In [1]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import csv
import datetime
from itertools import compress
import os
import re
import time
import zipfile
import  pytz  
import math
import string
import contractions
import matplotlib.pyplot as plt
from collections import Counter

##sklearn metrices and models
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


##torch and transformers
import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    MT5ForConditionalGeneration,
    PreTrainedTokenizer,
    T5TokenizerFast as T5Tokenizer,
    MT5TokenizerFast as MT5Tokenizer,
)
from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer                         
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import pipeline

##gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel,Word2Vec

##nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag
from nltk.corpus import wordnet

##Spacy
import spacy
from spacy import displacy
import en_core_web_sm
nltk.download('omw-1.4')


C:\softwares\conda4.8.2-python3.7\lib\site-packages\spacy\util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (2.3.0) was trained with spaCy v2.3 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\samra\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [2]:
data_path = 'data/raw/'

In [3]:
df_can_lst  = pd.read_csv(data_path + 'krs_canvas.csv')['KR_TEXT'].dropna().tolist()
df_lang_lst = pd.read_csv(data_path + 'krs_langmodel.csv')['Line'].dropna().tolist()
df_topic_lst = pd.read_csv(data_path + 'krs_topic.csv')['KR_NAME'].dropna().tolist()

In [4]:
def filter_client(line):
    if  ('clientname' in line or  
         'client name'in line or
        'client_name'in line or
        'clientnames' in line or
        'client names'in line or
        'client_names'in line) :
            
            return True
    else:
        return False

In [5]:
df_fltrd_topic_lst = [txt for txt in df_topic_lst if not filter_client(txt)]

In [6]:
text_List = df_can_lst +  df_fltrd_topic_lst + df_lang_lst

In [7]:
df = pd.DataFrame(text_List,columns =['raw_data'])
df = df.sample(frac = 1)
df = df.head(10000)

In [8]:
df.head()

,raw_data
203318,c. heated shield concept test for IF particle ...
269894,""""
394385,Update Vault page with Sales-facing assets (mo...
27076,(2025) OEM technology portfolio to support 202...
153850,Q3: KR - Help marketing launch 3 webinar serie...


In [9]:
import random
random.shuffle(text_List)

In [10]:
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
#nltk.download('all')
stop_words= stopwords.words('english')



In [11]:
def remove_quotes(text):
    punct = ''
    text = text.replace("'","")
    text = text.replace("^\"|\"$", "");
    return text

In [12]:
def decontract(text):
    expanded_words = []   
    for word in text.split():     
          expanded_words.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_words)
    return expanded_text


In [13]:
# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
tag_lst = ["CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNP","NNPS","NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB"]
def filter_punc_ascii(txt):

    text_arr = word_tokenize(txt)
    pos_lg = pos_tag(text_arr)
    new_txt_arr = []
    for item in pos_lg:
        if item[1] in tag_lst:
                new_txt_arr.append(item[0])
    return " ".join(new_txt_arr)

In [14]:
def clear_special_punc (text):
    punct = 'ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬ÃƒÂ¢Ã¢â‚¬Å¾Ã‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã¢â‚¬Å“ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€šÃ‚Â:""#$\\^`{|}~\t\n;ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬ÃƒÂ¢Ã¢â€šÂ¬Ã…â€œ``'''
    for p in punct:
        text = text.replace(p, '')
        text = text.strip()
    return text 

In [15]:
from multiprocessing import Pool
from functools import partial
import numpy as np

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)


In [17]:
#parallelize_on_rows(df, is_valid_comment)

In [20]:
def clean_data(text):
    
    text = remove_quotes(text)
    text = decontract(text)
    text = filter_punc_ascii(text)
    text = clear_special_punc(text)
    
    return text

In [21]:
#The comments should contain at least 2 different meaningful english words
def is_valid_comment(text):
    words = set(nltk.corpus.words.words())
    text_arr = set(word_tokenize(text))
    count=0
    for word in list(text_arr):
        if word in words:
            count = count + 1
    if count >=3:
        return True
    else:
        return False

In [22]:
df['clean_data'] = df['raw_data'].apply(lambda x : clean_data(x))

In [23]:
df['valid_clean_data'] =   df['clean_data'].apply(lambda x : is_valid_comment(x) )                                         

In [24]:
df = df[df['valid_clean_data']==True]

In [25]:
len(df)

8503

In [26]:
# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags: 
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

In [27]:
df['postag'] = df['clean_data'].apply(token_stop_pos)

In [28]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos: 
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:  
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

In [29]:
df['lemma'] = df['postag'].apply(lemmatize)

In [30]:
def sentiWordNetScore(pos_data):
    
    sentiment = 0
    tokens_count = 0
    status = ""
    for word, pos in pos_data:
        if not pos:
            continue
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        if not lemma:
            continue
        
        synsets = wordnet.synsets(lemma, pos=pos)
        if not synsets:
            continue
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())        
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        tokens_count += 1
        
    return sentiment
    

In [31]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\samra\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [32]:
df['swn'] = df['postag'].apply(sentiWordNetScore)

In [34]:
#!pip install vaderSentiment

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# function to calculate vader sentiment  
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs

df['vader'] = df['lemma'].apply(vadersentimentanalysis)

def get_vader_Neu(x_dict):
    return x_dict['neu']
    
def get_vader_Neg(x_dict):
    return x_dict['neg']

def get_vader_Pos(x_dict):
    return x_dict['pos']

def get_vader_Com(x_dict):
    return x_dict['compound']

df['vader_Neu'] = df['vader'].apply(get_vader_Neu)
df['vader_Pos'] = df['vader'].apply(get_vader_Pos)
df['vader_Neg'] = df['vader'].apply(get_vader_Neg)
df['vader_Com'] = df['vader'].apply(get_vader_Com)

In [36]:
import torch
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [37]:
def bert_Analysis(text):
    det = classifier(text)[0]
    score = np.round(det['score'],4)
    label = det['label']
    return label,score

In [38]:
def bert_classify(row):
    text = row['raw_data']
    det = classifier(text)[0]    
    return det

In [39]:
def bert_score(det):    
    score = np.round(det['score'],4)
    return score

In [40]:
def bert_label(det):    
    label = det['label']
    return label

In [41]:
df['bert'] = df.apply(bert_classify,axis=1)

In [42]:
df['bert_score']= df['bert'].apply(lambda x : bert_score(x))

In [43]:
df['bert_label']= df['bert'].apply(lambda x : bert_label(x))

In [79]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [80]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
# sdf = df.drop(['raw_data','postag','lemma','vader','bert','valid_clean_data'],axis=1)
# sdf.to_csv('data/sentiments_rules.csv')

In [48]:
#xdf = df.drop(['swn','vader_Neu','vader_Pos','vader_Neg','vader_Com','bert_label','bert_score'],axis=1)

In [49]:
#xdf.to_csv('data/sentiments_datas.csv')

In [83]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [85]:
def getRobertaVal(text):
    arr = []
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        arr.append({l,s})
    return arr    

In [92]:
df['roberta'] = df['clean_data'].apply(lambda x : getRobertaVal(x))

In [90]:
specific_model = pipeline("sentiment-analysis",model="finiteautomata/bertweet-base-sentiment-analysis")

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


In [96]:
specific_model(['Revisit communications for most commonly resolved ticket type and propose newly improved standard communication Roberto'])

[{'label': 'NEU', 'score': 0.8218857049942017}]

In [110]:
def get_bertweet(x):   
    try:
        data = specific_model([x])
    except:
        data = []
    print(" sentence is : ",x)
    return data
    
    

In [111]:
df['bertweet'] = df['clean_data'].apply(lambda x : get_bertweet(x))

 sentence is :  c. heated shield concept test for IF particle improvement
 sentence is :  Update Vault page with Sales-facing assets monthly and publish 2 articles in My Field News to drive 15 % increase in page views or asset downloads
 sentence is :  2025 OEM technology portfolio to support 2025 growth target 50M
 sentence is :  Q3 KR Help marketing launch 3 webinar series in Q3
 sentence is :  Execute prerequisites necessary for Exchange Archive Solution Launch 1 Identify Storage Requirements and Design 2 Define Training End User and Support and Communications 3 Leadership Approval 4 End User focus group conducted to align and ID any concerns with approach
 sentence is :  Increase the Support we offer to MIST Account Managers by 30 % in EMEA by increasing the number of quotes we create for no touch business i.e take on the partner lead MIST quoting
 sentence is :  To identify & amp publish the list of surplus manpower for redeployment in consultation with Business Leaders based on S

 sentence is :  Improvement on Retention % age of New joinees & lt 3 months
 sentence is :  Plan in place to drive Sales enablement achieved through Config Convertor and Bluecoat Migration Tools approved by CNS leadership
 sentence is :  Baseline Partner created references by end of Q1 Define Q2 goal
 sentence is :  Deliver at least 2 metrics based on agreed definitions in ACP suspect quality framework
 sentence is :  Successful Launch of Workday Learning Center as measured by continuity to the WE business over 5k external logins and a full billing cycle completed
 sentence is :  Establish new internal benchmark for global unsubscribe rate by 7/31
 sentence is :  Reduction in Renewal Processing Time by X %
 sentence is :  [ PLACEHOLDER ] Focus groups and panels for inclusion outreach efforts delivered
 sentence is :  [ Move to events OKR ] Create a visual verbal and video brand expressions and engaging experiences for branded internal and external events Sales Kick Off Spring PLS
 sent

 sentence is :  Complete Module 3 Natural language processing in business
 sentence is :  LMS/Certification to be at 89.9 % from 82.9 %
 sentence is :  Increase the number of Xfinity Assistant customers with a relevant predictive card measured by % without contact containment from 12 % to 20 % while achieving a click-through rate above 40 %
 sentence is :  Track progress against these identified applications
 sentence is :  Successful deployment of Wex in South Africa France Germany Spain and Italy
 sentence is :  All account managers exceed Q4 Juniper Mist forecast
 sentence is :  ZX Test Yield Fallout must be reduced Currently System Level 8T 20-30 % 16T 30-50 %
 sentence is :  Incease the Closure Rate of complaints with in TAT by 16 %
 sentence is :  Finalize Agendia customer data services boiler plate agreement by Jul 12 2019
 sentence is :  Increase teams understanding of how the work they do contributes to the org
 sentence is :  Baseline cost per ticket service desk NOC Eng
 sen

 sentence is :  Power compass with graph analytics and machine learning
 sentence is :  • Plan for a remote Annual Report process and begin engagement with stakeholders internal & amp external timetable scheduling and drafting
 sentence is :  Integration testing with RE changes as required
 sentence is :  100 % of teams who have been provided access to WorkBoard review OKRs weekly
 sentence is :  Provide resources and communication on when People Managers can send letters communicating rewards to employees during the annual Planning cycle by June 18 2020
 sentence is :  Top 3 use cases for each pillar are documented and 5 samples to create DB are available to customers for analyst persona
 sentence is :  Keep number of bugs in production less than 20
 sentence is :  Release one Pulse survey to baseline If there was one thing you could change or improve about the firm  s technology what would that be Internal survey share results with everyone
 sentence is :  Include accessibility link 

 sentence is :  Define and Document a plan for how to get logs into centralized system
 sentence is :  Drive number of completed BOD assessments to enable readiness on primary facilities for the next 90 day targeted quarter As measured by 1-50 % San Jose reports from Sept. 51-100 % develop WES/inhouse scope for 2020
 sentence is :  Activation of [ placeholder 263K ] in net new revenue for Q4 2021 expansion L1 KR
 sentence is :  Decrease 30 % in open balances by end of quater for MMM
 sentence is :  JMD Traffic Safety 2020-2021-Q41.1 Creating & amp Maintaining one conflict Management sheet for each Site Carried Forward 1 april 2020 to March 2021
 sentence is :  Meet all the Financial objectives of the Company a 100 % of SOX/Statutory Compliance b Meet OPEX FC accuracy BQ- 98 % & amp c 95 % of L3 SLAs
 sentence is :  Update MAC & amp iOS product troubleshooting training materials twice
 sentence is :  Org Structure realized with 100 % consistent operating metrics & amp budget in place ac

...

 sentence is :  100 % of functional DEI KRs are achieved and alignment within P & amp P DE & amp I frameworks
 sentence is :  Establish Asset share as our primary tool for content delivery to increase efficiency and promote cost savings across marketing
 sentence is :  Complete Sectigo ACME training for SSL Request team and onboard 2 stakeholders to the ACME protocol
 sentence is :  Filing strategy and related budgetary forecast for protection of innovation through patents in relevant global markets is shared with and approved by RK & amp Sean
 sentence is :  Commercial NGS support average responses time & lt 48 hrs monthly
 sentence is :  Deliver MVP of independent tooling to CDT team as beta user by Q3
 sentence is :  20 % decrease in repeat preventable interactions
 sentence is :  Plan to transition shared services from Finance team to Cesium GS finalized within 30 days of spin-off
 sentence is :  KR 1 Right-size labor hours to within -5 to +3 % versus ops-based targets in Power BI


 sentence is :  KR UK Activity transitioned to Solutions Delivery and Support between May 15 and May 30
 sentence is :  100 % of all internal tickets will be rated for Input Feedback
 sentence is :  Redemption issue ticket closure rate reduced from 8 days to within 5 days Monday to Friday
 sentence is :  100 % of all new non-whale projects should be handled by project partners
 sentence is :  Deliver 173 Cruisers to Delta to start module qualification starting 5/31
 sentence is :  Achieve gross revenue in Q1 of X toward FY21 goal of 355M
 sentence is :  SecureX is top 5 most viewed security hub page BDM AAG and FAQ – bonus top 5 unique visitors in dCloud demos
 sentence is :  Low Temp Heater for MOx determine the process operating window and validate dechuck at 200C without TPT OH penalty Jan
 sentence is :  Updating debug analysis for failing tests in Regression [ With in 24 hours of regression run ]
 sentence is :  Thermal SH CIP Qualify final design in lab Mar
 sentence is :  Build 

 sentence is :  Mature inventory of availability and governance of SLAs across all enterprise tech stacks
 sentence is :  Create a local field execution model to land CAF by 02/29
 sentence is :  Minimum of 50 check ins with crews or tenants on how SFS is doing or what can we do for them notes input with either tail numbers or tenant name and feedback escalated or reported as necessary
 sentence is :  Improve page performance of slowest pages time spent waiting by 10 %
 sentence is :  Deliver internal self-service reporting for Seismic APIM usage by September 30 [ WILL NOT DO ]
 sentence is :  Take initiatives on integrations by analyzing and quantifying integration systems to identify and address the perf issues proactively
 sentence is :  Number of users with claim grows to 200 in September
 sentence is :  Add addition features and modules to our Partner Portal for a more streamlined partner experience Focusing on the following Enable Lead Sharing for MSPs Partner Locator Journey Bui

 sentence is :  Create assessment and conduct partner audit for Hong Kong and Taiwan PSS partners
 sentence is :  Improved and clear billing and client friendly policies in place by end of January
 sentence is :  & lt 3 % differential in belonging index across generation
 sentence is :  Determine where approver list view dashboard resides
 sentence is :  Evangelize customers with concept of CEM by taking two deep dives presentations
 sentence is :  Demonstrate similarity performance for 4 new models e.g LSA pLSA LDA Bag of Words/TF * IDF simple Word2Vec for item description relative to existing syntactic string distance methods within one common framework
 sentence is :  Build out Product Styleguide/Icon library Setup library create email/newsletter design templates create icons maintain templates maintain product color palette and typography
 sentence is :  100 % of PLs complete Inclusive Hiring course as part of VIBE Way
 sentence is :  Juniper to be added into porfolio for AI Driven

 sentence is :  Develop an MVP minimal viable prototype for an experimental repository
 sentence is :  Run an in-app test to understand opp size for a financially sustainable cash-pay coach + PCP program
 sentence is :  Grow Cisco SPaaCH bookings by activating 5000 sites for ngena and X sites for MSX
 sentence is :  Closure of feature gaps in Regression suites
 sentence is :  Roll out and train bottoms-up segment SA planning to all theatre finance team
 sentence is :  Develop running list of product enhancements reviewed with Product issue resolution AHA
 sentence is :  Q3 Baseline costs for existing features specifically the 3 most common prompts worker sup org job profile
 sentence is :  Lead Engineers engagement to collaborate on Critical / Complex SRs for faster resolution
 sentence is :  Reduce failure rate to less than 1 % in PROD
 sentence is :  SonarQube report showing zero security vulnerabilities by August 30th tracked by % of fixed vulnerabilities
 sentence is :  LMS/Certifi

In [113]:
sdf = df.drop(['raw_data','postag','lemma','vader','bert','valid_clean_data'],axis=1)
sdf.to_csv('data/sentiments_rules.csv')

In [114]:
df

,raw_data,clean_data,valid_clean_data,postag,lemma,swn,vader,vader_Neu,vader_Pos,vader_Neg,vader_Com,bert,bert_score,bert_label,rating,roberta,bertweet
203318,c. heated shield concept test for IF particle ...,c. heated shield concept test for IF particle ...,True,"[(c., n), (heated, v), (shield, a), (concept, ...",c. heat shield concept test for IF particle ...,0.500,"{'neg': 0.0, 'neu': 0.727, 'pos': 0.273, 'comp...",0.727,0.273,0.000,0.4588,"{'label': 'NEGATIVE', 'score': 0.9750131368637...",0.9750,NEGATIVE,2,"[{0.8651923, Neutral}, {Positive, 0.0931992}, ...","[{'label': 'NEU', 'score': 0.9555402398109436}]"
394385,Update Vault page with Sales-facing assets (mo...,Update Vault page with Sales-facing assets mon...,True,"[(Update, n), (Vault, n), (page, n), (with, No...",Update Vault page with Sales-facing asset mo...,1.250,"{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'comp...",0.759,0.241,0.000,0.7430,"{'label': 'POSITIVE', 'score': 0.9378541707992...",0.9379,POSITIVE,2,"[{0.76669616, Neutral}, {Positive, 0.22060046}...","[{'label': 'NEU', 'score': 0.6077609062194824}]"
27076,(2025) OEM technology portfolio to support 202...,2025 OEM technology portfolio to support 2025 ...,True,"[(2025, None), (OEM, n), (technology, n), (por...",2025 OEM technology portfolio to support 202...,0.125,"{'neg': 0.0, 'neu': 0.602, 'pos': 0.398, 'comp...",0.602,0.398,0.000,0.6486,"{'label': 'POSITIVE', 'score': 0.9931983947753...",0.9932,POSITIVE,2,"[{Positive, 0.6719964}, {0.322131, Neutral}, {...","[{'label': 'POS', 'score': 0.5007180571556091}]"
153850,Q3: KR - Help marketing launch 3 webinar serie...,Q3 KR Help marketing launch 3 webinar series i...,True,"[(Q3, n), (KR, n), (Help, n), (marketing, n), ...",Q3 KR Help marketing launch 3 webinar series...,0.500,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.769,0.231,0.000,0.4019,"{'label': 'NEGATIVE', 'score': 0.9873908758163...",0.9874,NEGATIVE,2,"[{0.7768425, Neutral}, {Positive, 0.21810207},...","[{'label': 'NEU', 'score': 0.9675360918045044}]"
245800,"""Execute prerequisites necessary for Exchange ...",Execute prerequisites necessary for Exchange A...,True,"[(Execute, n), (prerequisites, v), (necessary,...",Execute prerequisites necessary for Exchange...,1.125,"{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compou...",0.820,0.180,0.000,0.7964,"{'label': 'NEGATIVE', 'score': 0.7661673426628...",0.7662,NEGATIVE,2,"[{0.89679193, Neutral}, {Positive, 0.0672121},...","[{'label': 'NEU', 'score': 0.9456313252449036}]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352792,"""Complete UAT, resolve critical defects, and d...",Complete UAT resolve critical defects and depl...,True,"[(Complete, a), (UAT, n), (resolve, v), (criti...",Complete UAT resolve critical defect and dep...,-0.500,"{'neg': 0.307, 'neu': 0.523, 'pos': 0.17, 'com...",0.523,0.170,0.307,-0.2732,"{'label': 'POSITIVE', 'score': 0.8766027092933...",0.8766,POSITIVE,2,"[{0.7837813, Neutral}, {Negative, 0.1480587}, ...","[{'label': 'NEU', 'score': 0.855009138584137}]"
297120,"""Stakeholder sign-off on 3-year roadmap for US...",Stakeholder sign-off on 3-year roadmap for US ...,True,"[(Stakeholder, n), (sign-off, n), (on, None), ...",Stakeholder sign-off on 3-year roadmap for U...,0.000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",1.000,0.000,0.000,0.0000,"{'label': 'NEGATIVE', 'score': 0.9388858079910...",0.9389,NEGATIVE,2,"[{0.8977384, Neutral}, {Positive, 0.08181638},...","[{'label': 'NEU', 'score': 0.9304724931716919}]"
420809,3.3. The Time to Interactive (TTI) for all LAT...,3.3 The Time to Interactive TTI for all LAT pa...,True,"[(3.3, None), (The, None), (Time, n), (to, Non...",3.3 The Time to Interactive TTI for all LAT ...,-0.375,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",1.000,0.000,0.000,0.0000,"{'label': 'NEGATIVE', 'score': 0.9974522590637...",0.9975,NEGATIVE,2,"[{0.8709365, Neutral}, {Positive, 0.09044363},...","[{'label': 'NEU', 'score': 0.8598824739456177}]"
100591,Identify major blocks o